# Create 2D horizontal plots
**Author: Jun Sasaki  Coded on 2025-01-13  Updated on 2025-01-14**<br>
Create a 2D horizontal contour plots. Customization can be made by defining `post_process_func`, or `post_process_func=None` without customization. 

```Python
def post_process_func(ax, da, time):
    """
    Example of post_process_func for customizing plot (e.g., add text or markers)
    
    Parameters:
    - ax: matplotlib axis.
    - da: DataArray (optional and used for dynamic customizing).
    - time: Frame time (optional and used for dynamic customizing).
    """
```

In [ ]:
import os
import pandas as pd
from xfvcom import FvcomDataLoader, FvcomPlotConfig, FvcomPlotter
from xfvcom.helpers import FrameGenerator

### Prepare FvcomDataLoader instance of `fvcom` using FVCOM output netcdf.
Dataset is `fvcom.ds`.

In [ ]:
# Loading FVCOM output netcdf
output_dir = "./"
output_dir = os.path.expanduser(output_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
base_path = "~/Github/TB-FVCOM/goto2023/output"
# List of netcdf files convenient to switch to another netcdf by specifying its index
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
index_ncfile = 0
# Create an instance of FvcomDataLoader where fvcom.ds is a Dataset
fvcom = FvcomDataLoader(base_path=base_path, ncfile=ncfiles[index_ncfile], time_tolerance=5)
cfg = FvcomPlotConfig(figsize=(6, 8))
plotter = FvcomPlotter(fvcom.ds, cfg)

### Create 2D plot with `post_process_plot`.
- 2-D horizontal plot with customization by updating `ax` if `post_process_plot` is defined.  

In [ ]:
def static_custom_plot(ax):
    """
    Customizing plot by updating ax

    Parameters:
    - ax: matplotlib axis
    """

    # Further customization can be added.
    ax.set_title("Title with Custom Plot")

def dynamic_custom_plot(ax, da, time):
    """
    Plot the corresponding datetime at each frame

    Parameters:
    - ax: matplotib axis.
    - da: DataArray.
    - time: Frame time.    
    """
    datetime = pd.Timestamp(da.time.item()).strftime('%Y-%m-%d %H:%M:%S')
    ax.set_title(f"Time: {datetime}")

# Specify var_name and siglay if any
var_name = "salinity"
time = 20
siglay = 0
da = plotter.ds[var_name][:,siglay,:] # time must be included in da as reusing a tool for animation.
time_str = fvcom.ds.time.isel(time=time).values
time_str = pd.to_datetime(time_str).strftime("%Y%m%d")
# Set plot_kwargs for `ax.tricontourf(**kwargs)`.
plot_kwargs={"verbose": False, "vmin": 10, "vmax": 20, "levels": [9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 15, 16, 17, 18, 19, 20]}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet"}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet", "with_mesh": True, "plot_grid": True, "add_tiles": True}
#plot_kwargs={}
save_path = os.path.join(output_dir, f"{var_name}_{time_str}.png")

ax = FrameGenerator.plot_data(data_array=da, time=time, plotter=plotter,
    save_path=None, post_process_func=dynamic_custom_plot, plot_kwargs=plot_kwargs)
fig=ax.figure
fig.savefig("test10.png")

### Specify xlim and ylim
Suppose `fvcom` and `plotter` instances, `var_name`, `siglay`, `time`, and `save_path` are prepared above.

In [ ]:
from xfvcom.helpers_utils import apply_xlim_ylim
import cartopy.crs as ccrs

In [ ]:
def custom_plot(ax, da, time):
    """
    Plot the corresponding datetime at each frame and set xlim and ylim

    Parameters:
    - ax: matplotib axis.
    - da: DataArray.
    - time: Frame time.
    """

    # Put datetime text.
    datetime = pd.Timestamp(da.time.item()).strftime('%Y-%m-%d %H:%M:%S')
    ax.set_title(f"Time: {datetime}")

    # Set xlim and ylim manually.
    xlim = ("139:40:00", "140:00:00")
    ylim = ("35:12:00", "35:50")
    xlim = (139.7, 140.1)
    ylim = (35, 35.6)

    apply_xlim_ylim(ax, xlim, ylim, is_cartesian=False)

# Set plot_kwargs for `ax.tricontourf(**kwargs)`.
plot_kwargs={"verbose": False, "vmin": 10, "vmax": 20, "levels": [9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 15, 16, 17, 18, 19, 20]}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet"}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet", "with_mesh": True, "projection": ccrs.PlateCarree()}
plot_kwargs={"verbose": False, "vmin": 28, "vmax": 34, "levels": 20, "cmap": "jet", "with_mesh": True, "projection": ccrs.Mercator()}

FrameGenerator.plot_data(data_array=da, time=time, plotter=plotter,
    save_path=save_path, post_process_func=custom_plot, plot_kwargs=plot_kwargs)